https://www.kaggle.com/c/platesv2

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import copy

import shutil 
from tqdm import tqdm

from torchvision import transforms, models

data_root = './plate'
print(os.listdir(data_root))

### Переносим тренировочные данные для последующего расширения их количества

In [2]:
TRAIN_DIR = 'train'
TEST_DIR = 'test'
class_names = ['cleaned', 'dirty']

# Создает новую директорию для используемых файлов
for class_name in class_names:
    os.makedirs(os.path.join(TRAIN_DIR, class_name), exist_ok=True)

# Копирует файлы в новую директорию
for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        dest_dir = os.path.join(TRAIN_DIR, class_name) 
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 2023.91it/s]


### Расширяем пространство тренировочных данных

In [3]:
MEAN = np.array([0.485, 0.456, 0.406])
STD = np.array([0.229, 0.224, 0.225])

train_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=90, fill=255),
    transforms.CenterCrop(180),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(hue=(0.1, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD)
])

train_dataset = torchvision.datasets.ImageFolder(TRAIN_DIR, train_transforms)

batch_size = 8
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size
)

In [4]:
# Указывается количество 
len(train_dataloader), len(train_dataset)

(5, 40)

### Обучающая функция

In [6]:
def train_model(model, dataloader, loss, optimizer, scheduler, num_epochs):
    accuracies = {'train': []}
    losses = {'train': []}
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training phase
        phase = 'train'
        model.train()  # Set model to training mode

        running_loss = 0.
        running_acc = 0.

        # Iterate over data
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Forward and backward
            with torch.set_grad_enabled(True):
                preds = model(inputs)
                loss_value = loss(preds, labels)
                preds_class = preds.argmax(dim=1)

                loss_value.backward()
                optimizer.step()

            # Statistics
            running_loss += loss_value.item()
            running_acc += (preds_class == labels.data).float().mean()
        
        scheduler.step()
            
        epoch_loss = running_loss / len(dataloader)
        epoch_acc = running_acc / len(dataloader)

        accuracies[phase].append(epoch_acc)
        losses[phase].append(epoch_loss)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

    return model, losses, accuracies

### Создание и обучение модели

In [18]:
import random

seed = 21
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(model.fc.in_features, 2)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # SGD

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [19]:
model, losses, accuracies = train_model(model, train_dataloader, loss,
                                        optimizer, scheduler, num_epochs=40)

Epoch 0/39:


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.26it/s]

train Loss: 0.8709 Acc: 0.3500
Epoch 1/39:



100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.32it/s]

train Loss: 0.6822 Acc: 0.5500
Epoch 2/39:


...

train Loss: 0.3974 Acc: 0.9000
Epoch 38/39:



100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.21it/s]

train Loss: 0.4099 Acc: 0.8500
Epoch 39/39:



100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.17it/s]

train Loss: 0.3931 Acc: 0.9250


### Графическое отображение точности и ошибки

In [20]:
plt.rcParams['figure.figsize'] = (10, 5)
for experiment_id in accuracies.keys():
    plt.plot(accuracies[experiment_id], label=experiment_id)
plt.legend()
plt.title('Accuracy')

"\nplt.rcParams['figure.figsize'] = (10, 5)\nfor experiment_id in accuracies.keys():\n    plt.plot(accuracies[experiment_id], label=experiment_id)\nplt.legend()\nplt.title('Accuracy')\n"

In [21]:
plt.rcParams['figure.figsize'] = (10, 5)
for experiment_id in losses.keys():
    plt.plot(losses[experiment_id], label=experiment_id)
plt.legend()
plt.title('Loss')

"\nplt.rcParams['figure.figsize'] = (10, 5)\nfor experiment_id in losses.keys():\n    plt.plot(losses[experiment_id], label=experiment_id)\nplt.legend()\nplt.title('Loss')\n"

### Некоторая обработка тестовых данных перед предсказанием

In [22]:
# Класс, который отдает путь к каждому изображению

class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [1]:
# Копируем пакпу test в рабочую область
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

### Обрезание тестовых изображений для лучшего качества предсказаний

In [23]:
test_transforms = transforms.Compose([
    transforms.CenterCrop(180),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD)
])

test_dataset = ImageFolderWithPaths(os.path.join("test"), transform=test_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=8, shuffle=False, num_workers=0
)

### Предсказание на тестовых данных

In [24]:
model.eval()

test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(paths)

test_predictions = np.concatenate(test_predictions)

100%|██████████████████████████████████████████████████████████████████████████████████| 93/93 [00:05<00:00, 17.60it/s]


### Запись результатов в файл

In [25]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

In [26]:
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.63 else 'cleaned')
submission_df.id = list(map(lambda x: x[-1], submission_df.id.str.split(r"\\")))
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)

In [27]:
submission_df.to_csv(os.path.join(data_root, 'submission.csv'))